# This notebook offers some experiments on the paper:


"Globally and Locally Consistent Image Completion", SATOSHI IIZUKA, EDGAR SIMO-SERRA,
HIROSHI ISHIKAWA

The employed code comes from https://github.com/akmtn/pytorch-siggraph2017-inpainting

It requires a PyTorch version below 1.0.

In [ ]:
%load_ext autoreload
%autoreload 2

from src.models import _NetCompletion, _NetContext, completionnet_places2
from src.ablation import completionnet_ablation, copy_weights
from src.masking import run_draw
from src.inpaint import inpainting, inpainting2, load_network, random_mask
from src.inpaint import load_mask, load_data, post_processing
from src.train import load_dataset, train_random_mask, train_discriminator, get_networks
import torchvision.transforms as transforms
from torch.nn.modules.loss import BCELoss, MSELoss
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import cv2
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import os
import urllib
import numpy as np

model, datamean = load_network()
filename = "completionnet_places2.t7"
url = "http://hi.cs.waseda.ac.jp/~iizuka/data/completionnet_places2.t7"

if not os.path.isfile(filename):
    urllib.urlretrieve(url, filename)

# 1. Simple tests on an image
Code from https://stackoverflow.com/a/36382158/4986615

Press ESC to quit the windows.

In [ ]:
img = cv2.imread("images/bridge.jpg")
run_draw(img, "mask.png")

In [ ]:
plt.figure()
mask = cv2.imread("mask.png")
plt.imshow(mask)
plt.axis('off')
plt.title("Mask")
plt.show()

In [ ]:
M = load_mask("mask.png", output_shape=(600, 400))
I = load_data("images/bridge.jpg", output_shape=(600, 400))
out = inpainting(model, datamean, I, M, postproc=True)
vutils.save_image(out, 'out.png', normalize=True)

In [ ]:
plt.figure(figsize=(20,10))
out_im = cv2.imread("out.png")[:,:,::-1] 
plt.imshow(out_im)
plt.axis('off')
plt.title("Output")
plt.show()

# 2. Computation of the loss function on a set of random masks
Here we want to draw a metric for the quality of the reconstruction.
We use a sum of a weighted MSE and a binary cross entropy as in the reference paper for training the generator.

In [ ]:
wtl2 = 0.5
bce_loss = BCELoss()
mse_loss = MSELoss()
        
M = random_mask(output_shape=(600, 400))
I = load_data("images/bridge.jpg", output_shape=(600, 400))
out = inpainting(model, datamean, I, M, postproc=False)
out2 = out.float()      

error = wtl2*mse_loss(out2, I) + (1 - wtl2)*bce_loss(out2, I)
print("Normal:", error)

out_proc = post_processing(I, M, out)
out_proc2 = out_proc.float()      
error = wtl2*mse_loss(out_proc2, I) + (1 - wtl2)*bce_loss(out_proc2, I)
print("Post-processing:", error)

# 3. Neural networks

The local and global discriminators were not open-sourced. They are implemented in `models.py`

In [ ]:
completion = _NetCompletion()
summary(completion, input_size=(4, 512, 512))

In [ ]:
from src.models import _NetContext
context = _NetContext()
summary(context, [(3, 128, 128), (3, 256, 256)])

# 4. Training

## 4.1. Visualize mask and patch
(Violet = global, green = local, yellow = hole)

In [ ]:
from src.train import train_random_mask
mask, patch = train_random_mask(2)
m, p = mask[0,0], patch[0]
m[p[0]:p[2], p[1]:p[3]] += 1
plt.imshow(m.numpy())
plt.axis('off')
plt.show()

## 4.2. Training the discriminator

In [ ]:
_, context = get_networks(cuda=False)
model, datamean = load_network()
dataloader = load_dataset(dataset="cifar10", dataroot="dataset/cifar10", batch_size=2)
train_discriminator(model, context, dataloader)

# 5. Removing neurons in the pre-trained model
Kind of an ablation study

In [ ]:
dropout = 0.1
A = completionnet_places2
A.load_state_dict(torch.load('completionnet_places2.pth'))
B = completionnet_ablation(dropout)
copy_weights(A, B)

# activate dropout during eval
B.eval()
for m in B.modules():
    if m.__class__.__name__.startswith('Dropout'):
        m.train()